# Calcul du ratio énergie/poids 

In [9]:
import pandas as pd
import numpy as np

In [10]:
#Récupération des données utiles
data_population = pd.read_csv('fr_population.csv')
data_animaux = pd.read_csv('fr_animaux.csv')
data_vegetaux = pd.read_csv('fr_vegetaux.csv')

#On gère nos noms de colonnes via des variables pour faciliter la maintenance du code
code_zone = "Code zone"
zone = 'Zone'
valeur = 'Valeur'
element = "Élément"
code_domaine = 'Code Domaine'
domaine = 'Domaine'
code_element = 'Code Élément'
code_produit = 'Code Produit'
produit = 'Produit'
code_annee = 'Code année'
annee = 'Année'
symbole = 'Symbole'
description_symbole = 'Description du Symbole'
unite = 'Unité'
energie_poids = 'Ratio énergie/poids en Kcal/Kg'
proteines_poids = 'Ratio proteines/poids en g/Kg'

data_population = data_population[data_population[code_zone] != 351]
data_animaux = data_animaux[data_animaux[code_zone] != 351]
data_vegetaux = data_vegetaux[data_vegetaux[code_zone] != 351]

In [11]:
#Fonction qui récupère la population d'un pays donné
def recup_pop(pays) :
    pop = data_population[data_population[zone] == pays][valeur].values
    pop = int(pop * 1000)
    return pop

In [12]:
#Fonction qui permet de calculer le ratio energie/poids d'un produit dans un pays
def calculer_energie_poids(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité alimentaire (Kcal/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    kcal = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            kcal = v
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                kcal *= pop * 365
                ratio = round((kcal / kg), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, energie_poids])
    data_ratio = data_ratio[data_ratio[energie_poids] != 0]
    return data_ratio
            
        

In [13]:
#Fonction qui permet de calculer le ratio poids de proteine/poids total
def calculer_ratio_proteines(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité de protéines en quantité (g/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    prot = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            prot = v /1000
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                prot *= pop * 365
                ratio = round(((prot / kg) * 100), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, proteines_poids])
    data_ratio = data_ratio[data_ratio[proteines_poids] != 0]
    return data_ratio

In [14]:
data_ratio_kcal = calculer_energie_poids(data_animaux)
data_ratio_kcal.head()

,Zone,Produit,Ratio énergie/poids en Kcal/Kg
0,Afghanistan,Viande de Bovins,2150.64
1,Afghanistan,Viande d'Ovins/Caprins,2304.64
2,Afghanistan,Viande de Volailles,1338.18
3,Afghanistan,"Viande, Autre",929.29
4,Afghanistan,Abats Comestible,1052.03


In [15]:
data_ratio_prot = calculer_ratio_proteines(data_vegetaux)
data_ratio_prot.head()

,Zone,Produit,Ratio proteines/poids en g/Kg
0,Afghanistan,Blé,8.41
1,Afghanistan,Riz (Eq Blanchi),7.13
2,Afghanistan,Orge,9.90
3,Afghanistan,Maïs,8.22
4,Afghanistan,Millet,7.43


In [16]:
#data_ratio_moyen = data_ratio.groupby(produit).mean(energie_poids)

In [17]:
#data_ratio_moyen.head()

In [18]:
data_ratio_kcal.to_csv('P3_04_01_données.csv')
data_ratio_prot.to_csv('P3_04_02_données.csv')